# Analiza ofert wynajmu mieszkań we Wrocławiu na podstawie danych serwisu OTODOM
## Problem Biznesowy
Problem biznesowy który zdecydowaliśmy się przeanalizować, polegał na sprawdzeniu, które z następujących udogodnień (bądź ich kombinacji) jest najbardziej opłacalna dla potencjalnego własciciela, przy wynajmie mieszkania; Które z następujących udogodnień, najbardziej podnosi cenę najmu mieszkania:
* Zmywarka
* Lodowka
* Meble
* Piekarnik
* Kuchenka
* Pralka
* Telewizor
* Telewizja kablowa
* Internet
* Telefon

## Metoda analizy
Dane zostały przeanalizowane przez model regresji liniowej, używając modelu liniowego z biblioteki *scikit-learn* - model oblicza współczynnik zmiany ceny. Dane zostały wczytane z pliku .csv do struktury *dataframe* z biblioteki *pandas*. W ramach analizy przeanalizowaliśmy następująco:
* Wzrost ceny w zależności od udogodnienia.
* Wzrost ceny za $m^2$ w zależności od udogodnienia.
* Wzrost ceny w zależności od kombinacji 2 udogodnień.
* Wzrost ceny za $m^2$ w zależności od kombinacji 2 udogodnień.

W celu analizy ceny za metr kwadratowy do ramki danych dodaliśmy kolumnę *cena_za_m2*. Z ramki danych usunęliśmy wszystkie pozycje które mogły miec pustą cenę/ powierzchnię. Stwierdziliśmy że warto także zliczać ilość występowania udogodnień oraz ich kombinacji.

In [8]:
import pandas as pd
import itertools
from sklearn.linear_model import LinearRegression

df = pd.read_csv("otodom_all.csv", sep=';', engine='python')

df = df.dropna(subset=['cenaPLN', 'powierzchnia_mkw'])
df['cena_za_m2'] = df['cenaPLN'] / df['powierzchnia_mkw']

features = [
    'zmywarka', 'lodowka', 'meble', 'piekarnik', 'kuchenka',
    'pralka', 'telewizor', 'telewizja_kablowa', 'internet', 'telefon'
]

freq = {}
for feature in features:
    freq[feature] = df[feature].sum()

freq

{'zmywarka': np.int64(2344),
 'lodowka': np.int64(3260),
 'meble': np.int64(3878),
 'piekarnik': np.int64(2730),
 'kuchenka': np.int64(2915),
 'pralka': np.int64(3111),
 'telewizor': np.int64(1602),
 'telewizja_kablowa': np.int64(1262),
 'internet': np.int64(2229),
 'telefon': np.int64(477)}

### Kombinacje udogodnień
Istnieje $\left( \begin{array}{c} 10 \\ 2 \end{array} \right) = \frac{10!}{2!(10-2)!} = \frac{10 * 9}{2} = 45$ kombinacji dwóch udogodnień. Żeby nie musieć spisywać ręcznie każdej z nich, uzyliśmy biblioteki *itertools* w celu wygenerowania wszystkich kombinacji automatycznie. Z każdej z nich powstała nowa kolumna zawierająca wynik mnożenia boolowskiego: 

***df['telefon_telewizor']*** = ***df['telefon']*** * ***df['telewizor']***

Pozwala to uzyskać współczynnik zmiany dla mieszkań które zawierają taką kombinację udogodnień. Najlepsze wyniki uzyskuje się licząc oddzielnie 4 warianty przypadków, wymienionych wyżej.


In [9]:
pairs = list(itertools.combinations(features, 2))

pair_freq = {}
for f1, f2 in pairs:
    colname = f"{f1}__{f2}"
    df[colname] = df[f1] * df[f2]
    pair_freq[colname] = df[colname].sum()

pair_freq

{'zmywarka__lodowka': np.int64(2312),
 'zmywarka__meble': np.int64(2252),
 'zmywarka__piekarnik': np.int64(2185),
 'zmywarka__kuchenka': np.int64(2204),
 'zmywarka__pralka': np.int64(2231),
 'zmywarka__telewizor': np.int64(1244),
 'zmywarka__telewizja_kablowa': np.int64(736),
 'zmywarka__internet': np.int64(1226),
 'zmywarka__telefon': np.int64(270),
 'lodowka__meble': np.int64(3158),
 'lodowka__piekarnik': np.int64(2693),
 'lodowka__kuchenka': np.int64(2905),
 'lodowka__pralka': np.int64(3096),
 'lodowka__telewizor': np.int64(1597),
 'lodowka__telewizja_kablowa': np.int64(903),
 'lodowka__internet': np.int64(1689),
 'lodowka__telefon': np.int64(322),
 'meble__piekarnik': np.int64(2637),
 'meble__kuchenka': np.int64(2847),
 'meble__pralka': np.int64(3035),
 'meble__telewizor': np.int64(1592),
 'meble__telewizja_kablowa': np.int64(1053),
 'meble__internet': np.int64(1946),
 'meble__telefon': np.int64(393),
 'piekarnik__kuchenka': np.int64(2598),
 'piekarnik__pralka': np.int64(2602),
 'p

## Dane ofert
Dane na podstawie których robiliśmy analizę zawierały wiele nieużytych przez nas kolumn. Wszystkie użyte przez nas kolumny zawierały tylko wartości liczbowe - łatwe do zrozumienia przez prymitywny model nauczania maszynowego, którego użyliśmy w celu wykonania tej analizy. Udogodnienia oznaczone są 0 w przypadku gdy ich nie ma, oraz 1 w przypadku gdy są. Inne użyte przez nas kolumny (czynsz, metraż w $m^2$).

## Wpływ udogodnień na cenę


In [10]:
X = df[features]
y = df['cenaPLN']

model = LinearRegression()
model.fit(X, y)

results = pd.DataFrame({
    'udogodnienie': X.columns,
    'wplyw_na_cene': model.coef_,
    'liczba_wystapien': [freq[col] for col in X.columns]
}).sort_values(by='wplyw_na_cene', ascending=False)
results

,udogodnienie,wplyw_na_cene,liczba_wystapien
0,zmywarka,659.248632,2344
6,telewizor,306.666701,1602
8,internet,223.767328,2229
7,telewizja_kablowa,130.115195,1262
3,piekarnik,-65.108974,2730
9,telefon,-116.041124,477
5,pralka,-207.062346,3111
4,kuchenka,-258.584413,2915
1,lodowka,-402.037340,3260
2,meble,-449.099941,3878


Udogodnieniem najbardziej wpływającym na czynsze okazała się zmywarka, a najbardziej szkodliwym - lodówka. Najrzadszym z udogodnień okazał się telefon - może sprawić że wyniki będą bardzo przechylone w którymś kierunku - szczególnie w przypadku kombinacji.

## Wpływ udogodnień na cenę za $m^2$

In [11]:
X = df[features]
y = df['cena_za_m2']

model = LinearRegression()
model.fit(X, y)

results = pd.DataFrame({
    'udogodnienie': X.columns,
    'wplyw_na_cene_m2': model.coef_,
    'liczba_wystapien': [freq[col] for col in X.columns]
}).sort_values(by='wplyw_na_cene_m2', ascending=False)
results

,udogodnienie,wplyw_na_cene_m2,liczba_wystapien
1,lodowka,6.308641,3260
0,zmywarka,4.145745,2344
8,internet,3.568490,2229
6,telewizor,2.952334,1602
4,kuchenka,2.560517,2915
7,telewizja_kablowa,0.892410,1262
2,meble,-1.902229,3878
9,telefon,-2.576766,477
5,pralka,-3.831876,3111
3,piekarnik,-10.622430,2730


Odwrotną obserwację, w porównaniu do poprzednich można zobaczyć, gdy będziemy analizowac nie cenę, ale cenę za $m^2$. W tym wypadku okazuje się że to lodówka jest najbardziej opłacalnym udogodnieniem, w jakie można zainwestować, a zmywarka zajmuje 2 miejsce. Piekarnik przynosi dużą szkodę ponad 10 złotych na metr kwadratowy do czynszu. Takie wyniki mogą wynikać z częstszego występowania lodówek, szczególnie w przypadku mieszkań z niższymi czynszami.

## Wpływ kombinacji udogodnień na cenę

In [12]:
X = df[[f"{f1}__{f2}" for f1, f2 in pairs]]
y = df['cenaPLN']

model = LinearRegression()
model.fit(X, y)

results = pd.DataFrame({
    'para': X.columns,
    'wplyw_na_cene': model.coef_,
    'liczba_wystapien': [pair_freq[col] for col in X.columns]
}).sort_values(by='wplyw_na_cene', ascending=False)
results

,para,wplyw_na_cene,liczba_wystapien
38,pralka__telefon,1959.689328,302
42,telewizja_kablowa__internet,973.366872,1168
19,meble__pralka,957.169824,3035
35,pralka__telewizor,953.278460,1560
15,lodowka__internet,885.310188,1689
0,zmywarka__lodowka,551.138990,2312
3,zmywarka__kuchenka,479.990937,2204
14,lodowka__telewizja_kablowa,446.744221,903
29,piekarnik__telefon,310.584286,280
7,zmywarka__internet,288.383989,1226


Jak wspominałem wcześniej, telefon może przechylać wyniki - w obie strony. Najbardziej opłacalnymi kombinacjami udogodnień są:
* Pralka i Telefon
* Telewizja kablowa i internet
* Meble i Pralka
* Pralka i Telewizor
* Lodówka i Internet

A takie połączenia jak lodówka z telefonem, bądź meblami są w stanie spowodować dużą stratę na czynszu.

## Wpływ kombinacji udogodnień na cenę za $m^2$

In [13]:
X = df[[f"{f1}__{f2}" for f1, f2 in pairs]]
y = df['cena_za_m2']

model = LinearRegression()
model.fit(X, y)

results = pd.DataFrame({
    'para': X.columns,
    'wplyw_na_cene_m2': model.coef_,
    'liczba_wystapien': [pair_freq[col] for col in X.columns]
}).sort_values(by='wplyw_na_cene_m2', ascending=False)
results

,para,wplyw_na_cene_m2,liczba_wystapien
29,piekarnik__telefon,26.067584,280
15,lodowka__internet,23.868730,1689
20,meble__telewizor,11.974141,1592
11,lodowka__kuchenka,11.640692,2905
19,meble__pralka,7.996518,3035
25,piekarnik__pralka,7.754759,2602
42,telewizja_kablowa__internet,7.719167,1168
36,pralka__telewizja_kablowa,6.239043,876
9,lodowka__meble,5.614774,3158
26,piekarnik__telewizor,4.507348,1348


W przypadku sprawdzania cen za $m^2$ sytuacja opłacalności się zmienia: najbardziej opłacalne okazują się połączenia mebli z pralką albo telewizorem, oraz lodówki z internetem albo kuchenką. Najbardziej opłacalna okazała siępara piekarnika i telefonu. Zaskakującą jest ilość połączeń lodówki, które przynoszą stratę.